# LangChain Expression Language

https://python.langchain.com/docs/expression_language/

Explain the basic syntax of LangChain Expression Language, which uses the pipe symbol (|) to connect components. Each component represents a specific task or action.

To make it as easy as possible to create custom chains, LangChain implemented a `Runnable`` protocol. The Runnable protocol is implemented for most components. This is a standard interface, which makes it easy to define custom chains as well as invoke them in a standard way. The standard interface includes:

- `stream`: stream back chunks of the response
- `invoke`: call the chain on an input
- `batch`: call the chain on a list of inputs

These also have corresponding async methods:

- `astream`: stream back chunks of the response async
- `ainvoke`: call the chain on an input async
- `abatch`: call the chain on a list of inputs async
- `astream_log`: stream back intermediate steps as they happen, in addition to the final response

----

## The Runnable Protocol:

A unit of work that can be invoked, batched, streamed, transformed and composed.

All methods accept an optional config argument, which can be used to configure execution, add tags and metadata for tracing and debugging etc.

Runnables expose schematic information about their input, output and config via the input_schema property, the output_schema property and config_schema method.

The LangChain Expression Language (LCEL) is a declarative way to compose Runnables into chains. Any chain constructed this way will automatically have sync, async, batch, and streaming support.

The main composition primitives are `RunnableSequence` and `RunnableParallel`.

---

## RunnableSequence

RunnableSequence invokes a series of runnables sequentially, with one runnable’s output serving as the next’s input. Construct using the | operator or by passing a list of runnables to RunnableSequence.

In [58]:
from langchain.schema.runnable import RunnableLambda

print(type(RunnableLambda(lambda x: x + 1))) # <class 'langchain.schema.runnable.RunnableLambda'>

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
print(type(sequence)) # <class 'langchain.schema.runnable.RunnableSequence'>

print('\n\n---')
print(sequence.invoke(1)) # 4
sequence.batch([1, 2, 3]) # [4, 6, 8]

<class 'langchain.schema.runnable.base.RunnableLambda'>
<class 'langchain.schema.runnable.base.RunnableSequence'>


---
4


[4, 6, 8]

## RunnableParallel

The RunnableParallel, allows for multiple runnables to be invoked in parallel, construct using a dictionary of runnables to invoke in parallel.

In [59]:
# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}

{'mul_2': 4, 'mul_5': 10}

---------------

## Combining the output of multiple runnables into a single response

A sequence that contains a RunnableParallel constructed using a dict literal, this is then followed by a RunnableLambda that consumes the output of the RunnableParallel

In [60]:
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
} | RunnableLambda(lambda x: x['mul_2'] + x['mul_5'])
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}

14

## Branching Out With Conditional Logic:

The runnable is initialized with a list of (condition, runnable) pairs and a default branch.

When operating on an input, the first condition that evaluates to True is selected, and the corresponding runnable is run on the input.

If no condition evaluates to True, the default branch is run on the input.

In [63]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
                (lambda x: True, lambda x: x),
                (lambda x: isinstance(x, str), lambda x: x.upper()),
                (lambda x: isinstance(x, int), lambda x: x + 1),
                (lambda x: isinstance(x, float), lambda x: x * 2),
                lambda x: "goodbye",
            )

print(branch.invoke("hello"))
# branch.invoke(None) # "goodbye"

hello


## Combining A RunnableBranch with a RunnableSequence:

In [64]:
from langchain.schema.runnable import RunnableParallel

parallel = RunnableParallel({
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
})

# This is a dictionary, however it will be composed with other runnables when used in a sequence:
parallel_two = {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}

print(type(parallel)) # <class 'langchain.schema.runnable.RunnableParallel'>
print(type(parallel_two)) # <class 'dict'>

<class 'langchain.schema.runnable.base.RunnableParallel'>
<class 'dict'>


In [66]:
# Now let's compose it with the runnable branch:
branch = RunnableBranch(
                (lambda x: isinstance(x, int), lambda x: x + 1),
                (lambda x: isinstance(x, float), lambda x: x * 2),
                # You must provide a default value:
                (lambda x: x * 2)
)

sequence = RunnableLambda(lambda x: x + 1) | branch | parallel
sequence.invoke(1) # {'mul_2': 6, 'mul_5': 15}

{'mul_2': 6, 'mul_5': 15}

---

## Accessing Previous Values using RunnablePassThrough and ItemGetter

RunnablePassThrough is a value that passes through and allows you to accessing previous values in a chain.

---

## Prompt + Model

---

## RAG

---

## Multiple Chains

---

## Adding Memory

---

## Bing Runtime Args

---

## Use RunnableParallel/RunnableMap

---

## Configurable Runnables

---

## Run Arbitrary Functions

---

## Fallbacks

---

## Custom Generator Functions

---

## RouterRunnable